In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import StandardScaler

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import random

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# Set random seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [3]:
# df = pd.read_csv('/home/janz/PROJECT/Tabular_Datas/processed_data_after_2010.csv')
df = pd.read_csv('/home/janz/PROJECT/Tabular_Datas/Tabular_for_cnn_updated.csv')

In [4]:
# # Plotting the distribution of the rescaled target variable (Last_UCVA)
# plt.figure(figsize=(10, 6))
# plt.hist(df['Last_UCVA'], bins=50, edgecolor='k', alpha=0.7)
# plt.xlabel('Last_UCVA')
# plt.ylabel('Frequency')
# plt.title('Distribution of Last_UCVA (Rescaled)')
# plt.grid(True)
# plt.show()

# # Adjust display settings and print the full value counts for detailed distribution
# pd.set_option('display.max_rows', None)
# print(df['Last_UCVA'].value_counts().sort_index())
# pd.reset_option('display.max_rows')  # Reset to default after printing


In [5]:
print(list(df))
df['Last_UCVA'] = df['Last_UCVA'] * 10
df.drop(columns=['Last_Efficacy Index'], inplace=True)
df.drop(columns=['Surgery Date'], inplace=True)
df.drop(columns=['Op.Time'], inplace=True)
df.drop(columns=['Last_Target Sph'], inplace=True)
df.drop(columns=['Therapeutic Cont L'], inplace=True)
df.drop(columns=['Era'], inplace=True)
df.drop(columns=['Season'], inplace=True)
df.drop(columns=['Temp'], inplace=True)
df.drop(columns=['Humidity'], inplace=True)

['Ocular Treatment ID', 'File', 'Age', 'Gender', 'Eye', 'Dominant Eye', 'Pachymetry', 'PRE-OP Average K', 'Pre-op K  Axis min', 'UCVA', 'Subjective SEQ', 'Subjective Cyl Axis', 'Subjective BCVA', 'Surgery Date', 'Treatment SEQ', 'Treatment Param Axis', 'Opt Zo', 'Max Abl Depth', 'Treatment Type', 'Humidity', 'Temp', 'Op.Time', 'Micro', 'Ring', 'Stop', 'Head', 'Alchohol', 'PTK mmm', 'MZ sec', 'Therapeutic Cont L', 'Rotation Angle Degrees', 'Last_Target Sph', 'Last_UCVA', 'Merge_Contact Lens', 'Season', 'Era', 'Last_Efficacy Index']


In [6]:
# Preprocess the data (convert non numeric data types to label encoder)
label_encoders = {}
for col in df.select_dtypes(include=['object']).columns:
    if col != "Ocular Treatment ID":
        label_encoders[col] = LabelEncoder()
        df[col] = label_encoders[col].fit_transform(df[col])

In [7]:
# # List of encoded treatment types
# treatment_types = [0, 1, 2]  # Femto Lasik, Lasik, PRK

# # Initialize dictionaries to hold train, validation, and test data (for each treatment type)
# train_data = {}
# val_data = {}
# test_data = {}

# set_seed(42)

# # Loop over each treatment type
# for treatment_type in treatment_types:
#     # Filter data for the current treatment type
#     treatment_data = df[df['Treatment Type'] == treatment_type]
    
#     # Split the data into train and test sets (80% train, 20% test)
#     train_data[treatment_type], test_data[treatment_type] = train_test_split(treatment_data, test_size=0.2, random_state=42)
    
#     # Further split the train data into train and validation sets (80% train, 20% validation)
#     train_data[treatment_type], val_data[treatment_type] = train_test_split(train_data[treatment_type], test_size=0.2, random_state=42)

# # Concatenate data from all treatment types
# train_data_all = pd.concat(train_data.values())
# val_data_all = pd.concat(val_data.values())
# test_data_all = pd.concat(test_data.values())

In [8]:
# Filter data based on Last_UCVA values
df_above = df[df['Last_UCVA'] > 0.8]
df_below = df[df['Last_UCVA'] <= 0.8]

# Split data above 0.8 for train and test (80-20 split)
train_data_above, test_data_above = train_test_split(df_above, test_size=0.2, random_state=42)

# Split data below/equal to 0.8 for train and test (80-20 split)
train_data_below, test_data_below = train_test_split(df_below, test_size=0.2, random_state=42)

# Further split train data above 0.8 into train and validation (80-20 split)
train_data_above, val_data_above = train_test_split(train_data_above, test_size=0.2, random_state=42)

# Further split train data below/equal to 0.8 into train and validation (80-20 split)
train_data_below, val_data_below = train_test_split(train_data_below, test_size=0.2, random_state=42)

# Concatenate data from all splits
train_data_all = pd.concat([train_data_above, train_data_below])
val_data_all = pd.concat([val_data_above, val_data_below])
test_data_all = pd.concat([test_data_above, test_data_below])

# Confirm the shapes of your datasets
print(f"Train data shape: {train_data_all.shape}")
print(f"Validation data shape: {val_data_all.shape}")
print(f"Test data shape: {test_data_all.shape}")

Train data shape: (34247, 28)
Validation data shape: (8562, 28)
Test data shape: (10703, 28)


In [9]:
# Drop the target columns and also Ocular Treatment ID, File columns
X_train = train_data_all.drop(columns=['Last_UCVA', 'Ocular Treatment ID', 'File'])
X_val = val_data_all.drop(columns=['Last_UCVA', 'Ocular Treatment ID', 'File'])
X_test = test_data_all.drop(columns=['Last_UCVA', 'Ocular Treatment ID', 'File'])

# # Extract the target variable
y_train = train_data_all['Last_UCVA']
y_val = val_data_all['Last_UCVA']
y_test = test_data_all['Last_UCVA']

In [10]:
# # Without scaling
# X_train_tensor = torch.tensor(X_train.values.astype(np.float32)).to(device)
# X_val_tensor = torch.tensor(X_val.values.astype(np.float32)).to(device)
# X_test_tensor = torch.tensor(X_test.values.astype(np.float32)).to(device)

# With scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train_scaled.astype(np.float32)).to(device)
X_val_tensor = torch.tensor(X_val_scaled.astype(np.float32)).to(device)
X_test_tensor = torch.tensor(X_test_scaled.astype(np.float32)).to(device)

# y tensors (does not matter if with or without scaling, because y labels will not be scaled)
y_train_tensor = torch.tensor(y_train.values.astype(np.float32)).unsqueeze(1).to(device)
y_val_tensor = torch.tensor(y_val.values.astype(np.float32)).unsqueeze(1).to(device)
y_test_tensor = torch.tensor(y_test.values.astype(np.float32)).unsqueeze(1).to(device)

In [11]:
# Hyperparameters
num_epochs = 100
batch_size = 64
learning_rate = 0.00001 # (high: 1 - 0.1)

In [12]:
# Data loaders
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=batch_size)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=batch_size)

In [ ]:
# class Net(nn.Module):
#     # Start by testing a small network built with layers of 100, 100, 30
#     def __init__(self):
#         super(Net, self).__init__()
#         self.fc1 = nn.Linear(X_train_tensor.shape[1], 100)
#         self.dropout1 = nn.Dropout(0.1)  # Add dropout layer with 20% dropout rate
#         self.fc2 = nn.Linear(100, 100)
#         self.dropout2 = nn.Dropout(0.05)  # Add dropout layer with 20% dropout rate
#         self.fc3 = nn.Linear(100, 30)
#         self.fc4 = nn.Linear(30, 1)
#         self.feature_extractor = nn.Linear(30, 30)  # Assuming feature_extractor is needed

#     def forward(self, x):
#         x = F.relu(self.fc1(x))
#         x = self.dropout1(x)  # Apply dropout after the first hidden layer
#         x = F.relu(self.fc2(x))
#         x = self.dropout2(x)  # Apply dropout after the second hidden layer
#         x = F.relu(self.fc3(x))
#         features = self.feature_extractor(x)  # Extract intermediate features
#         output = self.fc4(x)
#         return features, output
    

In [13]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(X_train_tensor.shape[1], 64)  # Increased neurons
        self.dropout1 = nn.Dropout(0.185)
        self.fc2 = nn.Linear(64, 128)  # Increased neurons
        self.dropout2 = nn.Dropout(0.185)
        self.fc3 = nn.Linear(128, 256)  # Increased neurons
        self.dropout3 = nn.Dropout(0.185)
        self.fc4 = nn.Linear(256, 128)  # Increased neurons
        self.dropout4 = nn.Dropout(0.185)
        self.fc5 = nn.Linear(128, 64)  # Increased neurons
        self.fc6 = nn.Linear(64, 1)
        self.feature_extractor = nn.Linear(64, 64)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = F.relu(self.fc3(x))
        x = self.dropout3(x)
        x = F.relu(self.fc4(x))
        x = self.dropout4(x)
        x = F.relu(self.fc5(x))
        features = self.feature_extractor(x)
        output = self.fc6(x)
        return features, output

**TEST RESULTS PLAYGROUND**

In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score
from tabulate import tabulate# Define different threshold configurations for evaluation

set_seed(42)

# class Net(nn.Module):
#     def __init__(self, input_size, dropout_rate):
#         super(Net, self).__init__()
#         self.fc1 = nn.Linear(input_size, 64)
#         self.dropout1 = nn.Dropout(dropout_rate)
#         self.fc2 = nn.Linear(64, 128)
#         self.dropout2 = nn.Dropout(dropout_rate)
#         self.fc3 = nn.Linear(128, 256)
#         self.dropout3 = nn.Dropout(dropout_rate)
#         self.fc4 = nn.Linear(256, 128)
#         self.dropout4 = nn.Dropout(dropout_rate)
#         self.fc5 = nn.Linear(128, 64)
#         self.fc6 = nn.Linear(64, 1)

#     def forward(self, x):
#         x = F.relu(self.fc1(x))
#         x = self.dropout1(x)
#         x = F.relu(self.fc2(x))
#         x = self.dropout2(x)
#         x = F.relu(self.fc3(x))
#         x = self.dropout3(x)
#         x = F.relu(self.fc4(x))
#         x = self.dropout4(x)
#         x = F.relu(self.fc5(x))
#         x = self.fc6(x)
#         return x
    
class Net(nn.Module):
    # Start by testing a small network built with layers of 100, 100, 30
    def __init__(self, input_size, dropout_rate):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 100)
        self.dropout1 = nn.Dropout(dropout_rate)  # Add dropout layer with 20% dropout rate
        self.fc2 = nn.Linear(100, 100)
        self.dropout2 = nn.Dropout(dropout_rate)  # Add dropout layer with 20% dropout rate
        self.fc3 = nn.Linear(100, 30)
        self.fc4 = nn.Linear(30, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)  # Apply dropout after the first hidden layer
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)  # Apply dropout after the second hidden layer
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Function for training the model
def train_model(config):
    # Unpack configuration
    num_epochs = config['num_epochs']
    batch_size = config['batch_size']
    learning_rate = config['learning_rate']
    dropout_rate = config['dropout_rate']

    # Initialize model, loss function, optimizer, and scheduler
    model = Net(input_size=X_train_tensor.shape[1], dropout_rate=dropout_rate).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)
    
    train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=batch_size)
    test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=batch_size)

    train_losses = []
    val_losses = []

    best_val_loss = float('inf')
    patience = 4
    epochs_without_improvement = 0

    for epoch in range(num_epochs):
        set_seed(42)
        print(f"Epoch {epoch} from {num_epochs}")
        model.train()
        running_train_loss = 0.0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item() * inputs.size(0)
        train_loss = running_train_loss / len(train_loader.dataset)
        train_losses.append(train_loss)

        model.eval()
        running_val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                running_val_loss += loss.item() * inputs.size(0)
        val_loss = running_val_loss / len(val_loader.dataset)
        scheduler.step(val_loss)
        val_losses.append(val_loss)

        if  val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement > patience:
                print(f'Early stopping at epoch {epoch}')
                break

    return model, {
        'train_loss': train_losses[-1],
        'val_loss': val_losses[-1]
    }

# Function for evaluating the model with different thresholds
def evaluate_model(model, config):
    test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=config['batch_size'])

    with torch.no_grad():
        total_mae = 0.0
        total_mse = 0.0
        all_predictions = []
        all_targets = []
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            total_mae += torch.mean(torch.abs(outputs - targets)).item()
            total_mse += torch.mean((outputs - targets) ** 2).item()
            all_predictions.extend(outputs.cpu().numpy().flatten())
            all_targets.extend(targets.cpu().numpy().flatten())
        mae = total_mae / len(test_loader)
        mse = total_mse / len(test_loader)

    all_predictions = np.array(all_predictions)
    all_targets = np.array(all_targets)
    abs_difference = np.abs(all_predictions - all_targets)
    abs_difference_mask = abs_difference <= config['abs_difference_threshold']

    binary_predictions = np.where(all_predictions >= config['binary_classification_threshold'], 1, 0)
    binary_targets = np.where(all_targets >= config['binary_classification_threshold'], 1, 0)
    for i in range(len(binary_predictions)):
        if abs_difference_mask[i]:
            binary_predictions[i] = binary_targets[i]

    precision = precision_score(binary_targets, binary_predictions)
    recall = recall_score(binary_targets, binary_predictions)
    accuracy = accuracy_score(binary_targets, binary_predictions)
    auc = roc_auc_score(binary_targets, binary_predictions)
    true_negatives = np.sum((binary_targets == 0) & (binary_predictions == 0))
    false_positives = np.sum((binary_targets == 0) & (binary_predictions == 1))
    false_alarm_rate = false_positives / (false_positives + true_negatives)

    return {
        'mae': mae,
        'mse': mse,
        'precision': precision,
        'recall': recall,
        'accuracy': accuracy,
        'auc': auc,
        'false_alarm_rate': false_alarm_rate,
        'config': config  # Include configuration in the results
    }

# Define the training configuration
training_config = {
    'num_epochs': 100,
    'batch_size': 64,
    'learning_rate': 0.00001,
    'dropout_rate': 0.185
}

# Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Assuming X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, X_test_tensor, y_test_tensor are already defined
trained_model, training_results = train_model(training_config)
print(f"Training results: {training_results}")

In [ ]:
# Define different threshold configurations for evaluation
threshold_configs = [
    {
        'binary_classification_threshold': 9.25,
        'abs_difference_threshold': 0,
        'dropout_rate': training_config['dropout_rate'],
        'batch_size': training_config['batch_size'],
        'num_epochs': training_config['num_epochs'],
        'learning_rate': training_config['learning_rate']
    },
    {
        'binary_classification_threshold': 9.25,
        'abs_difference_threshold': 1,
        'dropout_rate': training_config['dropout_rate'],
        'batch_size': training_config['batch_size'],
        'num_epochs': training_config['num_epochs'],
        'learning_rate': training_config['learning_rate']
    },
    {
        'binary_classification_threshold': 9,
        'abs_difference_threshold': 0,
        'dropout_rate': training_config['dropout_rate'],
        'batch_size': training_config['batch_size'],
        'num_epochs': training_config['num_epochs'],
        'learning_rate': training_config['learning_rate']
    },
    {
        'binary_classification_threshold': 9,
        'abs_difference_threshold': 1,
        'dropout_rate': training_config['dropout_rate'],
        'batch_size': training_config['batch_size'],
        'num_epochs': training_config['num_epochs'],
        'learning_rate': training_config['learning_rate']
    },
    {
        'binary_classification_threshold': 8.5,
        'abs_difference_threshold': 0,
        'dropout_rate': training_config['dropout_rate'],
        'batch_size': training_config['batch_size'],
        'num_epochs': training_config['num_epochs'],
        'learning_rate': training_config['learning_rate']
    },
    {
        'binary_classification_threshold': 8.5,
        'abs_difference_threshold': 1,
        'dropout_rate': training_config['dropout_rate'],
        'batch_size': training_config['batch_size'],
        'num_epochs': training_config['num_epochs'],
        'learning_rate': training_config['learning_rate']
    },
    {
        'binary_classification_threshold': 8,
        'abs_difference_threshold': 0,
        'dropout_rate': training_config['dropout_rate'],
        'batch_size': training_config['batch_size'],
        'num_epochs': training_config['num_epochs'],
        'learning_rate': training_config['learning_rate']
    },
    {
        'binary_classification_threshold': 8,
        'abs_difference_threshold': 1,
        'dropout_rate': training_config['dropout_rate'],
        'batch_size': training_config['batch_size'],
        'num_epochs': training_config['num_epochs'],
        'learning_rate': training_config['learning_rate']
    }
]

# threshold_configs = [
#     {
#         'binary_classification_threshold': 9.25,
#         'abs_difference_threshold': 0,
#         'batch_size': training_config['batch_size']
#     },
#     {
#         'binary_classification_threshold': 9.25,
#         'abs_difference_threshold': 1,
#         'batch_size': training_config['batch_size']
#     },
#     {
#         'binary_classification_threshold': 9,
#         'abs_difference_threshold': 0,
#         'batch_size': training_config['batch_size']
#     },
#     {
#         'binary_classification_threshold': 9,
#         'abs_difference_threshold': 1,
#         'batch_size': training_config['batch_size']
#     },
#     {
#         'binary_classification_threshold': 8.5,
#         'abs_difference_threshold': 0,
#         'batch_size': training_config['batch_size']
#     },
#     {
#         'binary_classification_threshold': 8.5,
#         'abs_difference_threshold': 1,
#         'batch_size': training_config['batch_size']
#     },
#     {
#         'binary_classification_threshold': 8,
#         'abs_difference_threshold': 0,
#         'batch_size': training_config['batch_size']
#     },
#     {
#         'binary_classification_threshold': 8,
#         'abs_difference_threshold': 1,
#         'batch_size': training_config['batch_size']
#     }
# ]

# Evaluate the model with different thresholds
evaluation_results = []
for config in threshold_configs:
    results = evaluate_model(trained_model, config)
    evaluation_results.append(results)

# Convert results to DataFrame and print as table
results_df = pd.DataFrame(evaluation_results)
# Print configurations as well
results_df['config'] = results_df['config'].apply(lambda x: str(x))
print(tabulate(results_df, headers='keys', tablefmt='psql'))

In [14]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

set_seed(42)

# model, loss function, and optimizer
model = Net().to(device)
criterion = nn.MSELoss() # MSE (Mean Squared Error)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0) # Adam is used as an optimizer# Train the model (high: 0.1 - 0.001)

# Learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

# Save losses for further analysis
train_losses = []
val_losses = []

best_val_loss = float('inf')
patience = 6
epochs_without_improvement = 0

for epoch in range(num_epochs):
    set_seed(42)
    model.train()
    running_train_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        # outputs = model(inputs)
        features, outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_train_loss += loss.item() * inputs.size(0)
    train_loss = running_train_loss / len(train_loader.dataset)
    train_losses.append(train_loss)

    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            # outputs = model(inputs)
            features, outputs = model(inputs)
            loss = criterion(outputs, targets)
            running_val_loss += loss.item() * inputs.size(0)
    val_loss = running_val_loss / len(val_loader.dataset)
    scheduler.step(val_loss)
    val_losses.append(val_loss)

    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')

    if  val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement > patience:
            print(f'Early stopping at epoch {epoch}')
            break

Epoch [1/100], Training Loss: 81.3145, Validation Loss: 65.3991
Epoch [2/100], Training Loss: 24.1423, Validation Loss: 6.1267
Epoch [3/100], Training Loss: 6.7319, Validation Loss: 5.0616
Epoch [4/100], Training Loss: 5.9655, Validation Loss: 4.4986
Epoch [5/100], Training Loss: 5.4948, Validation Loss: 4.1400
Epoch [6/100], Training Loss: 5.1733, Validation Loss: 3.8936
Epoch [7/100], Training Loss: 4.9363, Validation Loss: 3.7148
Epoch [8/100], Training Loss: 4.7506, Validation Loss: 3.5793
Epoch [9/100], Training Loss: 4.5979, Validation Loss: 3.4736
Epoch [10/100], Training Loss: 4.4678, Validation Loss: 3.3890
Epoch [11/100], Training Loss: 4.3542, Validation Loss: 3.3200
Epoch [12/100], Training Loss: 4.2529, Validation Loss: 3.2631
Epoch [13/100], Training Loss: 4.1612, Validation Loss: 3.2155
Epoch [14/100], Training Loss: 4.0772, Validation Loss: 3.1751
Epoch [15/100], Training Loss: 4.0002, Validation Loss: 3.1404
Epoch [16/100], Training Loss: 3.9290, Validation Loss: 3.110

In [ ]:
import optuna
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score

set_seed(42)

# Define your loss function
criterion = nn.MSELoss()

def objective(trial):
    set_seed(42)
    # Define hyperparameters to be optimized
    lr = trial.suggest_float('lr', 1e-6, 1e-1, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128, 8])
    
    # Create dataset and dataloaders with the suggested batch size
    train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=batch_size)
    # test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=batch_size)

    set_seed(42)

    # Load pre-trained DenseNet model and modify the final layer
    model = Net().to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)

    
    # Define the learning rate scheduler if needed
    # scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
    set_seed(42)

    best_val_loss = float('inf')
    patience = 6
    epochs_without_improvement = 0

    # Training loop
    for epoch in range(30):
        set_seed(42)
        model.train()
        running_train_loss = 0.0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            # outputs = model(inputs)
            _, outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item() * inputs.size(0)
        train_loss = running_train_loss / len(train_loader.dataset)
        train_losses.append(train_loss)

        model.eval()
        running_val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                # outputs = model(inputs)
                _, outputs = model(inputs)
                loss = criterion(outputs, targets)
                running_val_loss += loss.item() * inputs.size(0)
        val_loss = running_val_loss / len(val_loader.dataset)
        val_losses.append(val_loss)

        if  val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement > patience:
                print(f'Early stopping at epoch {epoch}')
                break

    # Initialize lists to store predictions and true values
    all_predictions = []
    all_targets = []

    # Test the model
    model.eval()
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            # inputs = inputs.to(device) 
            _, outputs = model(inputs)
            outputs = outputs.cpu().numpy().flatten()
            targets = targets.cpu().numpy().flatten()
            # Store predictions and true values for evaluation
            all_predictions.extend(outputs)
            # print(all_predictions)
            all_targets.extend(targets)

    # Convert predictions and targets to numpy arrays
    all_predictions = np.array(all_predictions)
    all_targets = np.array(all_targets)

    binary_classification_threshold = 8.5
    abs_difference_threshold = 1

    # Check if the absolute difference is smaller than the threshold
    abs_difference = np.abs(all_predictions - all_targets)
    abs_difference_mask = abs_difference <= abs_difference_threshold

    # Convert to binary classification based on threshold
    binary_predictions = np.where(all_predictions >= binary_classification_threshold, 1, 0)
    # print(binary_predictions)
    binary_targets = np.where(all_targets >= binary_classification_threshold, 1, 0)
    # print(binary_targets)

    for i in range(len(binary_predictions)):
        if abs_difference_mask[i] == True:
            binary_predictions[i] = binary_targets[i]

    # Calculate precision, recall, accuracy, and AUC
    precision = precision_score(binary_targets, binary_predictions)
    recall = recall_score(binary_targets, binary_predictions)
    accuracy = accuracy_score(binary_targets, binary_predictions)
    auc = roc_auc_score(binary_targets, binary_predictions)

    print(f'Test Precision: {precision:.4f}, Recall: {recall:.4f}, Accuracy: {accuracy:.4f}, AUC: {auc:.4f}')

    # Calculate false alarm rate (FPR)
    # False alarm rate = false positives / (false positives + true negatives)
    true_negatives = np.sum((binary_targets == 0) & (binary_predictions == 0))
    false_positives = np.sum((binary_targets == 0) & (binary_predictions == 1))
    false_alarm_rate = false_positives / (false_positives + true_negatives)

    print(f'False Alarm Rate: {false_alarm_rate:.4f}')
    
    return auc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Get the best hyperparameters
print('Best trial:', study.best_trial.params)


In [ ]:
num_epochs = len(train_losses)  # Update to reflect the actual number of epochs trained, useful if early stopping was triggered

def smooth_curve(points, factor=0.8):
    """Smooths the curve for better visualization."""
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

# Optionally smooth the loss curves
train_losses_smooth = smooth_curve(train_losses)
val_losses_smooth = smooth_curve(val_losses)

plt.figure(figsize=(10, 5))  # Set the figure size for better readability
plt.plot(range(1, num_epochs + 1), train_losses_smooth, label='Training Loss')
plt.plot(range(1, num_epochs + 1), val_losses_smooth, label='Validation Loss')
plt.title('Training and Validation Loss Curves')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)  # Add grid
plt.legend()
plt.show()

In [15]:
# Calculate Mean Absolute Error (MAE)
with torch.no_grad():
    total_mae = 0.0
    for inputs, targets in test_loader:
        # outputs = model(inputs)
        features, outputs = model(inputs)
        total_mae += torch.mean(torch.abs(outputs - targets)).item()
    mae = total_mae / len(test_loader)

print(f'Mean Absolute Error (MAE): {mae:.4f}')

# Calculate Mean Squared Error (MSE)
with torch.no_grad():
    total_mse = 0.0
    for inputs, targets in test_loader:
        # outputs = model(inputs)
        features, outputs = model(inputs)
        total_mse += torch.mean((outputs - targets) ** 2).item()
    mse = total_mse / len(test_loader)

print(f'Mean Squared Error (MSE): {mse:.4f}')


Mean Absolute Error (MAE): 1.3123
Mean Squared Error (MSE): 3.0524


In [16]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score

# Initialize lists to store predictions and true values
all_predictions = []
all_targets = []

# Test the model
model.eval()
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        # inputs = inputs.to(device) 
        # outputs = model(inputs)
        features, outputs = model(inputs)
        outputs = outputs.cpu().numpy().flatten()
        targets = targets.cpu().numpy().flatten()
        # Store predictions and true values for evaluation
        all_predictions.extend(outputs)
        # print(all_predictions)
        all_targets.extend(targets)

# Convert predictions and targets to numpy arrays
all_predictions = np.array(all_predictions)
all_targets = np.array(all_targets)

binary_classification_threshold = 8.5
abs_difference_threshold = 1

# Check if the absolute difference is smaller than the threshold
abs_difference = np.abs(all_predictions - all_targets)
abs_difference_mask = abs_difference <= abs_difference_threshold

# Convert to binary classification based on threshold
binary_predictions = np.where(all_predictions >= binary_classification_threshold, 1, 0)
# print(binary_predictions)
binary_targets = np.where(all_targets >= binary_classification_threshold, 1, 0)
# print(binary_targets)

for i in range(len(binary_predictions)):
    if abs_difference_mask[i] == True:
        binary_predictions[i] = binary_targets[i]

# Calculate precision, recall, accuracy, and AUC
precision = precision_score(binary_targets, binary_predictions)
recall = recall_score(binary_targets, binary_predictions)
accuracy = accuracy_score(binary_targets, binary_predictions)
auc = roc_auc_score(binary_targets, binary_predictions)

print(f'Test Precision: {precision:.4f}, Recall: {recall:.4f}, Accuracy: {accuracy:.4f}, AUC: {auc:.4f}')

# Calculate false alarm rate (FPR)
# False alarm rate = false positives / (false positives + true negatives)
true_negatives = np.sum((binary_targets == 0) & (binary_predictions == 0))
false_positives = np.sum((binary_targets == 0) & (binary_predictions == 1))
false_alarm_rate = false_positives / (false_positives + true_negatives)

print(f'False Alarm Rate: {false_alarm_rate:.4f}')

Test Precision: 0.9206, Recall: 0.7749, Accuracy: 0.7617, AUC: 0.7388
False Alarm Rate: 0.2974


Test Precision: 0.9184, Recall: 0.7852, Accuracy: 0.7676, AUC: 0.7373

False Alarm Rate: 0.3106

In [17]:
# Save the model
mlp_model_path = "/home/janz/PROJECT/trained_models/mlp_model.pth"
torch.save(model.state_dict(), mlp_model_path)
print(f"MLP model saved to {mlp_model_path}")

MLP model saved to /home/janz/PROJECT/trained_models/mlp_model.pth


In [ ]:
print(all_predictions[:30])
print(all_targets[:30])

In [ ]:
# Plotting the histogram
plt.figure(figsize=(5, 3))
plt.hist(all_predictions, bins=50, edgecolor='k', alpha=0.7)
plt.title('Distribution of Predictions')
plt.xlabel('Predicted Value')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# Plotting the histogram
plt.figure(figsize=(5, 3))
plt.hist(all_targets, bins=50, edgecolor='k', alpha=0.7)
plt.title('Distribution of actual Targets')
plt.xlabel('Predicted Value')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
# Calculate and display the confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns


cm = confusion_matrix(binary_targets, binary_predictions)
print("Confusion Matrix:")
print(cm)

# Plot the confusion matrix using seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
thresh = 8
above = 0
lower = 0
# for i in df["Last_UCVA"]:
for i in all_targets:
    if i >= thresh:
        above += 1
    else:
        lower += 1
print(f"There is {above} samples which there target value are above the value of {thresh}")
print(f"There is {lower} samples which there target value are under the value of {thresh}")

In [ ]:
import shap
np.random.seed(42)

# Initialize the SHAP GradientExplainer
background = X_train_tensor[:]  # Adjust this based on your dataset size and diversity
gradient_explainer = shap.GradientExplainer(model, background)

# Choose a specific instance to explain
instance_to_explain = X_train_tensor[0:1]  # Using a slice to keep the tensor's dimension

# Calculate SHAP values for this instance
shap_values = gradient_explainer.shap_values(instance_to_explain)

# If model is a classifier and shap_values is a list, take the first element
if isinstance(shap_values, list):
    shap_values = shap_values[0]

# Reshape SHAP values to remove unnecessary dimensions
shap_values = shap_values.squeeze()

# Convert the instance to numpy array for visualization
instance_numpy = instance_to_explain.detach().cpu().numpy()

# Calculate the expected value (mean output over the background data)
with torch.no_grad():
    model.eval()
    background_predictions = model(background)
    expected_value = background_predictions.mean().item()


# print("SHAP values for the instance:")
# for name, value in zip(list(X_train.columns), shap_values):
#     print(f"{name}: {value:.4f}")
# print("Expected value of the model output:", expected_value)

# # Optionally print other diagnostics
# print("Shape of SHAP values:", shap_values.shape)
# print("Shape of the instance data:", instance_numpy.shape)


# Plotting the waterfall chart for the first prediction

shap.waterfall_plot(shap.Explanation(values=shap_values,  # Reshaped SHAP values
                                     base_values=expected_value,  # Computed expected value
                                     data=instance_numpy[0],  # The specific instance being explained
                                     feature_names=list(X_train.columns)), max_display=10)  # Actual feature names

shap_values = shap_values.reshape(1, -1)

shap.summary_plot(shap_values, instance_to_explain, plot_type="bar", feature_names=X_train.columns, max_display=10)

In [ ]:
import xgboost as xgb
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score

set_seed(42)

# Define a threshold for binary classification
binary_classification_threshold = 8
abs_difference_threshold = 0

# Define and train XGBoost model
xgb_model = xgb.XGBRegressor(n_estimators=100, use_label_encoder=False)
xgb_model.fit(X_train_scaled, y_train.values)

# Test XGBoost model
y_pred = xgb_model.predict(X_test_scaled)

# Check if the absolute difference is smaller than the threshold
abs_difference = np.abs(y_pred - y_test.values)
abs_difference_mask = abs_difference <= abs_difference_threshold

# Convert probabilities to binary predictions based on threshold
binary_predictions_xgb = np.where(y_pred >= binary_classification_threshold, 1, 0)
# Convert target variable to binary labels based on the threshold for test set
binary_y_test = np.where(y_test >= binary_classification_threshold, 1, 0)

for i in range(len(binary_predictions_xgb)):
    if abs_difference_mask[i] == True:
        binary_predictions_xgb[i] = binary_y_test[i]

# Calculate precision, recall, accuracy, and AUC for XGBoost model on test data
precision_xgb = precision_score(binary_y_test, binary_predictions_xgb)
recall_xgb = recall_score(binary_y_test, binary_predictions_xgb)
accuracy_xgb = accuracy_score(binary_y_test, binary_predictions_xgb)
auc_xgb = roc_auc_score(binary_y_test, binary_predictions_xgb)

true_negatives_xgb = np.sum((binary_y_test == 0) & (binary_predictions_xgb == 0))
false_positives_xgb = np.sum((binary_y_test == 0) & (binary_predictions_xgb == 1))
false_alarm_rate_xgb = false_positives_xgb / (false_positives_xgb + true_negatives_xgb)

# Print evaluation metrics for XGBoost model
print(f'XGBoost Test Precision: {precision_xgb:.4f}, Recall: {recall_xgb:.4f}, Accuracy: {accuracy_xgb:.4f}, AUC: {auc_xgb:.4f}')
print(f'False Alarm Rate: {false_alarm_rate_xgb:.4f}')

from sklearn.ensemble import RandomForestRegressor

# Define Random Forest model
rf_model = RandomForestRegressor(n_estimators=100)

# Train Random Forest model
rf_model.fit(X_train_scaled, y_train.values)

# Test Random Forest model
y_pred_rf = rf_model.predict(X_test_scaled)

# Check if the absolute difference is smaller than the threshold
abs_difference = np.abs(y_pred_rf - y_test.values)
abs_difference_mask = abs_difference <= abs_difference_threshold

# Convert probabilities to binary predictions based on threshold
binary_predictions_rf = np.where(y_pred_rf >= binary_classification_threshold, 1, 0)
# Convert target variable to binary labels based on the threshold for test set
binary_y_test = np.where(y_test >= binary_classification_threshold, 1, 0)

for i in range(len(binary_predictions_rf)):
    if abs_difference_mask[i] == True:
        binary_predictions_rf[i] = binary_y_test[i]

# Calculate precision, recall, accuracy, and AUC for Random Forest model on test data
precision_rf = precision_score(binary_y_test, binary_predictions_rf)
recall_rf = recall_score(binary_y_test, binary_predictions_rf)
accuracy_rf = accuracy_score(binary_y_test, binary_predictions_rf)
auc_rf = roc_auc_score(binary_y_test, binary_predictions_rf)

true_negatives_rf = np.sum((binary_y_test == 0) & (binary_predictions_rf == 0))
false_positives_rf = np.sum((binary_y_test == 0) & (binary_predictions_rf == 1))
false_alarm_rate_rf = false_positives_rf / (false_positives_rf + true_negatives_rf)

# Print evaluation metrics for Random Forest model
print(f'Random Forest Test Precision: {precision_rf:.4f}, Recall: {recall_rf:.4f}, Accuracy: {accuracy_rf:.4f}, AUC: {auc_rf:.4f}')
print(f'False Alarm Rate: {false_alarm_rate_rf:.4f}')

XGBoost Test Precision: 0.9017, Recall: 0.9524, Accuracy: 0.8704, AUC: 0.6688

False Alarm Rate: 0.6147

Random Forest Test Precision: 0.9072, Recall: 0.9450, Accuracy: 0.8702, AUC: 0.6864

False Alarm Rate: 0.5721

In [ ]:
import shap
np.random.seed(42)

# Create a SHAP explainer object using TreeExplainer for XGBoost model
explainer_xgb = shap.TreeExplainer(xgb_model)
shap_values_xgb = explainer_xgb.shap_values(X_test_scaled)

# Plotting the SHAP values for XGBoost model
shap.summary_plot(shap_values_xgb, X_test_scaled, plot_type="bar", feature_names=X_train.columns, max_display=10)

# Compute the mean SHAP values across all instances
mean_shap_values_xgb = np.mean(shap_values_xgb, axis=0)

# Plotting the waterfall plot for the aggregated SHAP values
shap.waterfall_plot(shap.Explanation(values=mean_shap_values_xgb, base_values=explainer_xgb.expected_value, data=X_test_scaled.mean(axis=0), feature_names=X_train.columns), max_display=10)


In [ ]:
# # Create a SHAP explainer object using TreeExplainer for Random Forest model
# explainer_rf = shap.TreeExplainer(rf_model)
# shap_values_rf = explainer_rf.shap_values(X_test_scaled)
# # Plotting the SHAP values for Random Forest model
# # Plotting the SHAP values for XGBoost model
# shap.summary_plot(shap_values_rf, X_test_scaled, plot_type="bar", feature_names=X_train.columns)

# # Compute the mean SHAP values across all instances
# mean_shap_values_xgb = np.mean(shap_values_rf, axis=0)

# # Plotting the waterfall plot for the aggregated SHAP values
# shap.waterfall_plot(shap.Explanation(values=mean_shap_values_xgb, base_values=explainer_xgb.expected_value, data=X_test_scaled.mean(axis=0), feature_names=X_train.columns), max_display=10)


In [ ]:
# # Set option to display all columns
# pd.set_option('display.max_columns', None)

# # Set option to display all rows
# pd.set_option('display.max_rows', None)